In [11]:
using Plots
include("GenerateGrid.jl")
include("MyEconFcn.jl")

Main.MyEconFcn

In [12]:
function crra(cons::Float64, γ::Float64)
    if γ != 1.0
        util = cons^(1-γ) / (1-γ)
    else
        util = log(cons)
    end
    return util
end

crra (generic function with 1 method)

In [2]:
struct Params
    # parameters
    β::Float64 # Discount factor (\beta)
    γ::Float64 # Relative risk aversion (\gamma)
    rent::Float64
    wH::Float64
    wL::Float64
    p1::Float64
    p2::Float64

    # grid
    na::Int64 # the number of grid points for capital
    amax::Float64 # the maximum value of the grid for capital
    amin::Float64 # the minimum value of the grid for capital
    agrid::Vector{Float64} # the grid for capital

    # iterations
    maxit::Int64 # the maximum number of iterations
    tol::Float64 # the tolerance of errors
end

In [7]:
function calibration()
    β = 0.985
    γ = 1.0
    rent = 1.01-1.0
    wH = 1.0
    wL = 0.1
    p1 = 0.9
    p2 = 0.9

    na = 501
    amax = 10.0 # If \delta is 0.08, make it 10.0
    amin = -1.0

    # Make equally distanced grid by own code
    agrid = GenerateGrid.grid_uni(amin, amax, na)
    # Or use it as before
    # kgrid = collect(LinRange(amin, amax, na))

    maxit = 10000
    tol = 1e-5 # set the tolerance of errors

    return Params(β, γ, rent, wH, wL, p1, p2, na, amax, amin, agrid, maxit, tol)
end

calibration (generic function with 1 method)

In [8]:
params = calibration();

# Define variables
vfcn = zeros(params.na) # intial guess of value function
pfcn = zeros(Int64,params.na)
Tvfcn = zeros(params.na)
Tpfcn = zeros(Int64,params.na)

# Variables for plot
val_tmp = zeros(params.na, 4)
dif = zeros(2, params.maxit);

In [17]:
# Initial values of the utility function (put penalty for negative consumption)
util = -10000.0*ones(params.na, params.na, 2)

# Compute the utility for combinations of (k,k') such that consumption is positive
for i = 1:params.na, j = 1:params.na
                    
    cH = (1.0+params.rent)*params.agrid[j] - params.agrid[i] + params.wH
    cL = (1.0+params.rent)*params.agrid[j] - params.agrid[i] + params.wL

    if cH > 0.0
        util[j,i,1] = crra(params.γ,cH)
    elseif cL > 0.0
        util[j,i,2] = crra(params.γ,cL)
    end

end

In [18]:
print(util[1,1,1])
print(params.wH)
# print(util[10,1,1])

99.999999999999911.0

In [20]:
# Iterations for maxit times at the maximum
for it = 1:params.maxit

    vkp1 = zeros(params.na, params.na)
    vkp2 = zeros(params.na, params.na)
            
    # Compute combinations of the value function
    for i = 1:params.na
        vkp1[:,i] = util[:,i,1] + params.β*(params.p1*vfcn[:,1]+(1-params.p1)*vfcn[:,2])
        vkp2[:,i] = util[:,i,2] + params.β*(params.p2*vfcn[:,1]+(1-params.p2)*vfcn[:,2])
    end

    # Look for savings to maximize the value function
    for i = 1:params.na
        Tvfcn[i,1] = maximum(vkp1[:, i])
        Tpfcn[i,1] = argmax(vkp1[:, i])
        Tvfcn[i,2] = maximum(vkp2[:, i])
        Tpfcn[i,2] = argmax(vkp2[:, i])
    end

    # Check the convergence: Check errors in interations
    dif1 = maximum(abs.((Tvfcn-vfcn)./vfcn))
    dif2 = maximum(abs.(Tpfcn-pfcn)./pfcn)

    # Save errors in interations before the convergence
    # Usually unnessesary as they are for the purpose of showing the iteration process
    dif[1, it] = dif1
    dif[2, it] = dif2

    # Usually unnessesary as they are for the purpose of showing the convergence of the value function
    if it==1 || it==3 || it==5
        val_tmp[:, count] = vfcn
        count = count + 1
    end

    # Update the value and policy functions
    vfcn = deepcopy(Tvfcn)
    pfcn = deepcopy(Tpfcn)

    println([it,dif1,dif2])
    
    # Check if converged
    if dif1 < params.tol
        val_tmp[:, end] = vfcn
        break
    end
    
    if it == params.maxit
        println("The model does not converge")
    end
end

LoadError: BoundsError: attempt to access 501-element Vector{Float64} at index [1:501, 2]